In [ ]:
import sklearn as sk
import numpy as np
import pandas as pd
!pip install thefuzz

In [ ]:
df=pd.read_csv("adult.csv")
df.columns = df.columns.str.replace('.', '_') #points in the headlines give errors


In [ ]:
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [ ]:
df.shape

(32561, 15)

In [ ]:
df.duplicated().sum() #counting duplicates

24

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [ ]:
df.describe()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [ ]:
df.isna().sum() # as it can be seen in head(), there are missings

,0
age,0
workclass,0
fnlwgt,0
education,0
education_num,0
marital_status,0
occupation,0
relationship,0
race,0
sex,0


In [ ]:
df_trimmed = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [ ]:
#every string to lower case - function
def to_lowercase(x):
    if isinstance(x, str):
        return x.lower()
    return x

In [ ]:
df_lowercase = df_trimmed.applymap(to_lowercase)
print(df_lowercase)

       age workclass  fnlwgt     education  education_num      marital_status  \
0       90         ?   77053       hs-grad              9             widowed   
1       82   private  132870       hs-grad              9             widowed   
2       66         ?  186061  some-college             10             widowed   
3       54   private  140359       7th-8th              4            divorced   
4       41   private  264663  some-college             10           separated   
...    ...       ...     ...           ...            ...                 ...   
32556   22   private  310152  some-college             10       never-married   
32557   27   private  257302    assoc-acdm             12  married-civ-spouse   
32558   40   private  154374       hs-grad              9  married-civ-spouse   
32559   58   private  151910       hs-grad              9             widowed   
32560   22   private  201490       hs-grad              9       never-married   

              occupation   

<ipython-input-13-b423b6d81d5b>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_lowercase = df_trimmed.applymap(to_lowercase)


In [ ]:
df_lowercase.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,90,?,77053,hs-grad,9,widowed,?,not-in-family,white,female,0,4356,40,united-states,<=50k
1,82,private,132870,hs-grad,9,widowed,exec-managerial,not-in-family,white,female,0,4356,18,united-states,<=50k
2,66,?,186061,some-college,10,widowed,?,unmarried,black,female,0,4356,40,united-states,<=50k
3,54,private,140359,7th-8th,4,divorced,machine-op-inspct,unmarried,white,female,0,3900,40,united-states,<=50k
4,41,private,264663,some-college,10,separated,prof-specialty,own-child,white,female,0,3900,40,united-states,<=50k


In [ ]:
#TheFuzz library is used to see if there are string mismatches
from thefuzz import fuzz
from thefuzz import process

In [ ]:
df_lowercase.native_country.unique()

array(['united-states', '?', 'mexico', 'greece', 'vietnam', 'china',
       'taiwan', 'india', 'philippines', 'trinadad&tobago', 'canada',
       'south', 'holand-netherlands', 'puerto-rico', 'poland', 'iran',
       'england', 'germany', 'italy', 'japan', 'hong', 'honduras', 'cuba',
       'ireland', 'cambodia', 'peru', 'nicaragua', 'dominican-republic',
       'haiti', 'el-salvador', 'hungary', 'columbia', 'guatemala',
       'jamaica', 'ecuador', 'france', 'yugoslavia', 'scotland',
       'portugal', 'laos', 'thailand', 'outlying-us(guam-usvi-etc)'],
      dtype=object)

In [ ]:
unique_native_country_count = df_lowercase['native_country'].nunique()
print(unique_native_country_count)

42


All (cat.) columns have unique values. Native_countries is checked due to multiple values

In [ ]:
df_lowercase['native_country'] = df_lowercase['native_country'].replace('?', np.nan) #'?' would lead to error

common_countries = df_lowercase['native_country'].value_counts().index.tolist()

def correct_spelling_fuzz(value, correct_list, score_threshold=90):
    if isinstance(value, str):
        # Find the closest match
        best_match = process.extractOne(value, correct_list)
        # If the similarity score is above the threshold, the spelling will be corrected
        if best_match and best_match[1] >= score_threshold:
            return best_match[0]
    return value

df2 = df_lowercase.copy()

# apply function
df2['native_country'] = df2['native_country'].apply(lambda x: correct_spelling_fuzz(x, common_countries))

print(df2)

       age workclass  fnlwgt     education  education_num      marital_status  \
0       90         ?   77053       hs-grad              9             widowed   
1       82   private  132870       hs-grad              9             widowed   
2       66         ?  186061  some-college             10             widowed   
3       54   private  140359       7th-8th              4            divorced   
4       41   private  264663  some-college             10           separated   
...    ...       ...     ...           ...            ...                 ...   
32556   22   private  310152  some-college             10       never-married   
32557   27   private  257302    assoc-acdm             12  married-civ-spouse   
32558   40   private  154374       hs-grad              9  married-civ-spouse   
32559   58   private  151910       hs-grad              9             widowed   
32560   22   private  201490       hs-grad              9       never-married   

              occupation   

In [ ]:
unique_native_country_count = df2['native_country'].nunique()
print(unique_native_country_count)      #no spelling mistakes, only unique values

41


In [ ]:
#drops rows that are identical (when all values in all the columns are the same)
df3 = df2.drop_duplicates()

In [ ]:
df3.shape # at the beginning: (32561, 15)

(32537, 15)

In [ ]:
#Mixed data types check
!pip install deepchecks

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.6 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 65.6 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.6 MB/s eta 0:00:00
  Created wheel for PyNomaly: filename=PyNomaly-0.3.3-py3-none-any.whl size=8479 sha256=acd8263d8c2c7ea5d13679aeb15d4c3ee8175ec6348333c0531b1862ef514c22
  Stored in directory: /root/.cache/pip/wheels/2b/49/68/62eb262833c0fd5403d3b258bd259b0f71516d2563a960303f
Successfully built PyNoma

In [ ]:
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import MixedDataTypes

In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32537 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32537 non-null  int64 
 1   workclass       32537 non-null  object
 2   fnlwgt          32537 non-null  int64 
 3   education       32537 non-null  object
 4   education_num   32537 non-null  int64 
 5   marital_status  32537 non-null  object
 6   occupation      32537 non-null  object
 7   relationship    32537 non-null  object
 8   race            32537 non-null  object
 9   sex             32537 non-null  object
 10  capital_gain    32537 non-null  int64 
 11  capital_loss    32537 non-null  int64 
 12  hours_per_week  32537 non-null  int64 
 13  native_country  31955 non-null  object
 14  income          32537 non-null  object
dtypes: int64(6), object(9)
memory usage: 4.0+ MB


In [ ]:
categorical_features = ['workclass', 'education', 'marital_status',
                        'occupation', 'relationship', 'race', 'sex', 'native_country']

#get a dataset out of cat. features

dataset = Dataset(df3, label=None, cat_features=categorical_features)

mixed_types_check = MixedDataTypes().run(dataset)

mixed_types_check.show()

<!--
 ~ ----------------------------------------------------------------------------
 ~ Copyright (C) 2021-2023 Deepchecks (https://www.deepchecks.com)
 ~
 ~ This file is part of Deepchecks.
 ~ Deepchecks is distributed under the terms of the GNU Affero General
 ~ Public License (version 3 or later).
 ~ You should have received a copy of the GNU Affero General Public License
 ~ along with Deepchecks. If not, see .
 ~ ----------------------------------------------------------------------------
 ~
-->
<!DOCTYPE html>
 
 
 
 Mixed Data Types

In [ ]:
check = MixedDataTypes().add_condition_rare_type_ratio_not_in_range((0.01, 0.2))
result = check.run(df3)
result.show(show_additional_outputs=False)

deepchecks - WARNING - Received a "pandas.DataFrame" instance. It is recommended to pass a "deepchecks.tabular.Dataset" instance by initializing it with the data and metadata, for example by doing "Dataset(dataframe, label=label, cat_features=cat_features)"
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 10 categorical features were inferred.: workclass, education, education_num, marital_status, occupation, relationship, race... For full list use dataset.cat_features


<!--
 ~ ----------------------------------------------------------------------------
 ~ Copyright (C) 2021-2023 Deepchecks (https://www.deepchecks.com)
 ~
 ~ This file is part of Deepchecks.
 ~ Deepchecks is distributed under the terms of the GNU Affero General
 ~ Public License (version 3 or later).
 ~ You should have received a copy of the GNU Affero General Public License
 ~ along with Deepchecks. If not, see .
 ~ ----------------------------------------------------------------------------
 ~
-->
<!DOCTYPE html>
 
 
 
 Mixed Data Types

No column has mixed datatypes.

In [ ]:
#checking for special characters
from deepchecks.tabular.checks import SpecialCharacters

In [ ]:
dataset = Dataset(df3, label=None)

special_characters_check = SpecialCharacters().run(dataset)

special_characters_check.show()

deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 10 categorical features were inferred.: workclass, education, education_num, marital_status, occupation, relationship, race... For full list use dataset.cat_features


<!--
 ~ ----------------------------------------------------------------------------
 ~ Copyright (C) 2021-2023 Deepchecks (https://www.deepchecks.com)
 ~
 ~ This file is part of Deepchecks.
 ~ Deepchecks is distributed under the terms of the GNU Affero General
 ~ Public License (version 3 or later).
 ~ You should have received a copy of the GNU Affero General Public License
 ~ along with Deepchecks. If not, see .
 ~ ----------------------------------------------------------------------------
 ~
-->
<!DOCTYPE html>
 
 
 
 Special Characters

The '?' stands for NaN and will be converted.



Next step: fill NaNs for one hot encoding




In [ ]:
df3.replace('?', np.nan, inplace=True)

In [ ]:
#df3.workclass.unique()
df3.occupation.unique()


array([nan, 'exec-managerial', 'machine-op-inspct', 'prof-specialty',
       'other-service', 'adm-clerical', 'craft-repair',
       'transport-moving', 'handlers-cleaners', 'sales',
       'farming-fishing', 'tech-support', 'protective-serv',
       'armed-forces', 'priv-house-serv'], dtype=object)

In [ ]:
#calculat skewness to see if the mean or median imputer should be used

from scipy.stats import skew
df4 = df3.dropna()


age_skewness = skew(df4['age'].dropna())
fnlwgt_skewness = skew(df4['fnlwgt'].dropna())
education_num_skewness = skew(df4['education_num'].dropna())
capital_gain_skewness = skew(df4['capital_gain'].dropna())
capital_loss_skewness = skew(df4['capital_loss'].dropna())
hours_per_week_skewness = skew(df4['hours_per_week'].dropna())

print(f"Skewness from 'age' column: {age_skewness}")
print(f"Skewness from 'fnlwgt' column: {fnlwgt_skewness}")
print(f"Skewness from 'education' column: {education_num_skewness}")
print(f"Skewness from 'capital_gain' column: {capital_gain_skewness}")
print(f"Skewness from 'capital_loss' column: {capital_loss_skewness}")
print(f"Skewness from 'hours_per_week' column: {hours_per_week_skewness}")


Skewness from 'age' column: 0.5289447607879796
Skewness from 'fnlwgt' column: 1.4599827714003706
Skewness from 'education' column: -0.30282983035773847
Skewness from 'capital_gain' column: 11.897512038029832
Skewness from 'capital_loss' column: 4.524183679463225
Skewness from 'hours_per_week' column: 0.33236942852231155



As from the results can be seen (Positive Skewness = Right-Skewed, Negative Skewness = Left-Skewed)

*   'age' column: 0.529 - Slightly positively skewed

*   'fnlwgt' column: 1.460 - Moderately positively skewed
*   'education' column: -0.303 - Slightly negatively skewed


*   'capital_gain' column: 11.898 - Extremely positively skewed

*   'capital_loss' column: 4.524 - Highly positively skewed


*   'hours_per_week' column: 0.332 - Slightly positively skewed

For the slightly skewed columns, median or mean could be used. As median imputation is the safer way, this imputation methode will be used for all columns.

In [ ]:
# checking for outliers
from deepchecks.tabular.checks import OutlierSampleDetection

In [ ]:
check = OutlierSampleDetection()
check.add_condition_outlier_ratio_less_or_equal(max_outliers_ratio=0.001, outlier_score_threshold=0.9)
check.run(df3)

deepchecks - WARNING - Received a "pandas.DataFrame" instance. It is recommended to pass a "deepchecks.tabular.Dataset" instance by initializing it with the data and metadata, for example by doing "Dataset(dataframe, label=label, cat_features=cat_features)"
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 10 categorical features were inferred.: workclass, education, education_num, marital_status, occupation, relationship, race... For full list use dataset.cat_features


<!--
 ~ ----------------------------------------------------------------------------
 ~ Copyright (C) 2021-2023 Deepchecks (https://www.deepchecks.com)
 ~
 ~ This file is part of Deepchecks.
 ~ Deepchecks is distributed under the terms of the GNU Affero General
 ~ Public License (version 3 or later).
 ~ You should have received a copy of the GNU Affero General Public License
 ~ along with Deepchecks. If not, see .
 ~ ----------------------------------------------------------------------------
 ~
-->
<!DOCTYPE html>
 
 
 
 Outlier Sample Detection

As there are many outliers, the rows which have outliers, detected by deepchecks, will be deleted.

In [ ]:
from scipy.stats import median_abs_deviation


numeric_cols = df3.select_dtypes(include=[np.number])

# Calculate the median and MAD for numeric columns
median = numeric_cols.median()
mad = numeric_cols.apply(median_abs_deviation)

# Calculation of z-score
z_scores = 0.6745 * (numeric_cols - median) / mad  # mit normalizing factor scores comparable to the standard z-scores


print(z_scores)

# rows which have Z-scores beyond the threshold
outliers_count = (np.abs(z_scores) > 3.5).sum(axis=1) #3.5 is common for robust z-scores
print(outliers_count.value_counts())

rows_to_keep = outliers_count == 0

# Filter the original DataFrame to keep only these rows
df3_cleaned = df3[rows_to_keep]

# Optionally, print the number of rows before and after filtering
print(f"Original number of rows: {df3.shape[0]}")
print(f"Number of rows after removing outliers: {df3_cleaned.shape[0]}")

           age    fnlwgt  education_num  capital_gain  capital_loss  \
0      3.57485 -1.141192        -0.6745           NaN           inf   
1      3.03525 -0.512406        -0.6745           NaN           inf   
2      1.95605  0.086798         0.0000           NaN           inf   
3      1.14665 -0.428041        -4.0470           NaN           inf   
4      0.26980  0.972260         0.0000           NaN           inf   
...        ...       ...            ...           ...           ...   
32556 -1.01175  1.484700         0.0000           NaN           NaN   
32557 -0.67450  0.889337         1.3490           NaN           NaN   
32558  0.20235 -0.270160        -0.6745           NaN           NaN   
32559  1.41645 -0.297918        -0.6745           NaN           NaN   
32560 -1.01175  0.260608        -0.6745           NaN           NaN   

       hours_per_week  
0            0.000000  
1           -4.946333  
2            0.000000  
3            0.000000  
4            0.000000  
...

Before imputation: split dataset

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df3_cleaned, test_size=0.25)

X_test = test_df.drop(columns=['income'])
y_test = test_df['income']

In [ ]:
X_test.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
7057,58,private,241056,some-college,10,divorced,adm-clerical,unmarried,white,female,0,0,46,united-states
25716,19,private,410543,hs-grad,9,never-married,adm-clerical,not-in-family,white,female,0,0,40,united-states
7060,59,federal-gov,31840,hs-grad,9,married-civ-spouse,craft-repair,husband,white,male,0,0,40,united-states
16247,52,private,99602,hs-grad,9,separated,craft-repair,own-child,black,female,0,0,40,united-states
16991,27,private,164924,hs-grad,9,never-married,craft-repair,not-in-family,white,male,0,0,40,united-states


In [ ]:
y_test.head()

,income
7057,<=50k
25716,<=50k
7060,>50k
16247,<=50k
16991,<=50k


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16274 entries, 7159 to 5314
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             16274 non-null  int64 
 1   workclass       15523 non-null  object
 2   fnlwgt          16274 non-null  int64 
 3   education       16274 non-null  object
 4   education_num   16274 non-null  int64 
 5   marital_status  16274 non-null  object
 6   occupation      15520 non-null  object
 7   relationship    16274 non-null  object
 8   race            16274 non-null  object
 9   sex             16274 non-null  object
 10  capital_gain    16274 non-null  int64 
 11  capital_loss    16274 non-null  int64 
 12  hours_per_week  16274 non-null  int64 
 13  native_country  16004 non-null  object
 14  income          16274 non-null  object
dtypes: int64(6), object(9)
memory usage: 2.0+ MB


In [ ]:
#median  imputation for numeric columns
from sklearn.impute import SimpleImputer


numerical_columns = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']


num_imputer = SimpleImputer(strategy='median')

# Apply the imputer directly to the numerical columns in the DataFrame
train_df[numerical_columns] = num_imputer.fit_transform(train_df[numerical_columns])


print("\nTrain-DataFrame:")
print(train_df)

### Test-data transformation###

#transform numerical columns
X_test_imputed = num_imputer.transform(X_test[numerical_columns])


X_test_imputed_df = pd.DataFrame(X_test_imputed, columns=numerical_columns, index=X_test.index)

# combination of cat colomns and imputed num columns. The indices stay the same as in original X_test
X_test_final = pd.concat([X_test.drop(columns=numerical_columns), X_test_imputed_df], axis=1)

# The columns are the same as in original X_test
X_test_final = X_test_final[X_test.columns]


print("\nTest-DataFrame:")
print(X_test_final.head())





Train-DataFrame:
        age         workclass    fnlwgt     education  education_num  \
7159   46.0  self-emp-not-inc   28281.0     bachelors           13.0   
32121  26.0               NaN  375313.0  some-college           10.0   
22533  44.0  self-emp-not-inc  219591.0  some-college           10.0   
11992  47.0           private   47496.0       hs-grad            9.0   
14745  39.0           private  187089.0  some-college           10.0   
...     ...               ...       ...           ...            ...   
21123  43.0         local-gov  105896.0  some-college           10.0   
29403  23.0           private  347292.0       hs-grad            9.0   
13658  25.0           private   19214.0       hs-grad            9.0   
28067  32.0         local-gov  114733.0     bachelors           13.0   
5314   49.0           private  148995.0       hs-grad            9.0   

           marital_status         occupation   relationship  \
7159   married-civ-spouse    farming-fishing        hu

In [ ]:
train_df.isna().sum()

,0
age,0
workclass,751
fnlwgt,0
education,0
education_num,0
marital_status,0
occupation,754
relationship,0
race,0
sex,0


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16274 entries, 7159 to 5314
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             16274 non-null  float64
 1   workclass       15523 non-null  object 
 2   fnlwgt          16274 non-null  float64
 3   education       16274 non-null  object 
 4   education_num   16274 non-null  float64
 5   marital_status  16274 non-null  object 
 6   occupation      15520 non-null  object 
 7   relationship    16274 non-null  object 
 8   race            16274 non-null  object 
 9   sex             16274 non-null  object 
 10  capital_gain    16274 non-null  float64
 11  capital_loss    16274 non-null  float64
 12  hours_per_week  16274 non-null  float64
 13  native_country  16004 non-null  object 
 14  income          16274 non-null  object 
dtypes: float64(6), object(9)
memory usage: 2.0+ MB


In [ ]:
from sklearn.impute import SimpleImputer

object_columns = ['workclass', 'education','marital_status', 'occupation', 'relationship', 'race', 'sex','native_country']

imputer = SimpleImputer(strategy='most_frequent')

# Apply the imputation
train_df[object_columns] = imputer.fit_transform(train_df[object_columns])

print("\nImputed Train-DataFrame:")
print(train_df)

###Test-dataset transformation###

# Imputation on categorical columns
X_test = imputer.transform(X_test_final[object_columns])

# dataframe with original indices from X_test_final
X_test_imputed_df = pd.DataFrame(X_test, columns=object_columns, index=X_test_final.index)

#original columns like in X_train_final
X_test_num = X_test_final[numerical_columns]

#combine numeric and imputes cat columns
X_test_final_imputed = pd.concat([X_test_num, X_test_imputed_df], axis=1)

#to make sure, that the columns are the same
X_test_final_imputed = X_test_final_imputed[X_test_final.columns]

print("\nFinal Imputed Test DataFrame:")
print(X_test_final_imputed.head())





Imputed Train-DataFrame:
        age         workclass    fnlwgt     education  education_num  \
7159   46.0  self-emp-not-inc   28281.0     bachelors           13.0   
32121  26.0           private  375313.0  some-college           10.0   
22533  44.0  self-emp-not-inc  219591.0  some-college           10.0   
11992  47.0           private   47496.0       hs-grad            9.0   
14745  39.0           private  187089.0  some-college           10.0   
...     ...               ...       ...           ...            ...   
21123  43.0         local-gov  105896.0  some-college           10.0   
29403  23.0           private  347292.0       hs-grad            9.0   
13658  25.0           private   19214.0       hs-grad            9.0   
28067  32.0         local-gov  114733.0     bachelors           13.0   
5314   49.0           private  148995.0       hs-grad            9.0   

           marital_status         occupation   relationship  \
7159   married-civ-spouse    farming-fishing  

In [ ]:
X_test_final_imputed.isna().sum()

,0
age,0
workclass,0
fnlwgt,0
education,0
education_num,0
marital_status,0
occupation,0
relationship,0
race,0
sex,0


In [ ]:
train_df.isna().sum()

,0
age,0
workclass,0
fnlwgt,0
education,0
education_num,0
marital_status,0
occupation,0
relationship,0
race,0
sex,0


In [ ]:
rows_with_nan = train_df[train_df.isna().any(axis=1)]

# Zeilen anzeigen, die NaN-Werte enthalten
print(rows_with_nan)

Empty DataFrame
Columns: [age, workclass, fnlwgt, education, education_num, marital_status, occupation, relationship, race, sex, capital_gain, capital_loss, hours_per_week, native_country, income]
Index: []


In [ ]:
#check for linearity - one-hot encoding to use linear regression
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.compose import ColumnTransformer

In [ ]:
categorical_columns = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']

encoder = OneHotEncoder(sparse_output=False) #dense array returned

# fit one-hot encoding to the categorical columns
one_hot_encoded = encoder.fit_transform(train_df[categorical_columns])

#get_feature_names_out() for getting the cat. columns names
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

#controlling, that the indices are the same
one_hot_df.index = train_df.index

# orig. dataframe with one-hot-encoded dataframe
df_encoded = pd.concat([train_df, one_hot_df], axis=1)

df_encoded = df_encoded.drop(categorical_columns, axis=1) #drop original cat. col.

print("\nEncoded data:")
print(df_encoded.head())


Encoded data:
        age    fnlwgt  education_num  capital_gain  capital_loss  \
7159   46.0   28281.0           13.0           0.0           0.0   
32121  26.0  375313.0           10.0           0.0           0.0   
22533  44.0  219591.0           10.0           0.0           0.0   
11992  47.0   47496.0            9.0           0.0           0.0   
14745  39.0  187089.0           10.0           0.0           0.0   

       hours_per_week income  workclass_federal-gov  workclass_local-gov  \
7159             40.0   >50k                    0.0                  0.0   
32121            40.0  <=50k                    0.0                  0.0   
22533            40.0  <=50k                    0.0                  0.0   
11992            42.0  <=50k                    0.0                  0.0   
14745            40.0  <=50k                    0.0                  0.0   

       workclass_never-worked  ...  native_country_portugal  \
7159                      0.0  ...                      

In [ ]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16274 entries, 7159 to 5314
Data columns (total 100 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   age                                        16274 non-null  float64
 1   fnlwgt                                     16274 non-null  float64
 2   education_num                              16274 non-null  float64
 3   capital_gain                               16274 non-null  float64
 4   capital_loss                               16274 non-null  float64
 5   hours_per_week                             16274 non-null  float64
 6   income                                     16274 non-null  object 
 7   workclass_federal-gov                      16274 non-null  float64
 8   workclass_local-gov                        16274 non-null  float64
 9   workclass_never-worked                     16274 non-null  float64
 10  workclass_private       

In [ ]:
print(X_test_final_imputed[categorical_columns].isna().sum())

workclass         0
education         0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
native_country    0
dtype: int64


In [ ]:
###Test-dataset transformation##
#one-hot-encoding

X_test_final_imputed_array = encoder.transform(X_test_final_imputed[categorical_columns])

# array to dataframe with original index from X_test_final_imputed
X_test_encoded_df = pd.DataFrame(X_test_final_imputed_array,
                                 columns=encoder.get_feature_names_out(categorical_columns),
                                 index=X_test_final_imputed.index)

#combine datasets, dropping the original cat columns and
X_test_final = pd.concat([X_test_final_imputed.drop(columns=categorical_columns), X_test_encoded_df], axis=1)

print("\nEncoded test data:")
print(X_test_final.head())


Encoded test data:
        age    fnlwgt  education_num  capital_gain  capital_loss  \
7057   58.0  241056.0           10.0           0.0           0.0   
25716  19.0  410543.0            9.0           0.0           0.0   
7060   59.0   31840.0            9.0           0.0           0.0   
16247  52.0   99602.0            9.0           0.0           0.0   
16991  27.0  164924.0            9.0           0.0           0.0   

       hours_per_week  workclass_federal-gov  workclass_local-gov  \
7057             46.0                    0.0                  0.0   
25716            40.0                    0.0                  0.0   
7060             40.0                    1.0                  0.0   
16247            40.0                    0.0                  0.0   
16991            40.0                    0.0                  0.0   

       workclass_never-worked  workclass_private  ...  \
7057                      0.0                1.0  ...   
25716                     0.0             

In [ ]:
df_encoded.income.unique()

array(['>50k', '<=50k'], dtype=object)

In [ ]:
#income should be a dummy variable of 0 and 1 - label encoding

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder



# LabelEncoder instanziieren
encoder = LabelEncoder()

# Fit the LabelEncoder on the 'income' column of the training data
encoder.fit(df_encoded['income'])

# Transform 'income' in the training data
df_encoded['income'] = encoder.transform(df_encoded['income'])

# Transform 'income' in the test data (y_test)
y_test_encoded = encoder.transform(y_test)

# y_test_encoded in einen DataFrame umwandeln und sicherstellen, dass die Indizes von y_test beibehalten werden
y_test_encoded_df = pd.DataFrame(y_test_encoded, columns=['income_encoded'], index=y_test.index)

# Ausgabe des DataFrames zur Überprüfung
print(y_test_encoded_df.head())




       income_encoded
7057                0
25716               0
7060                1
16247               0
16991               0


In [ ]:
y_test_encoded_df.shape

(5425, 1)

In [ ]:
y_test_encoded_df.head()

,income_encoded
7057,0
25716,0
7060,1
16247,0
16991,0


In [ ]:
df_encoded.income.unique()

array([1, 0])

In [ ]:
df_encoded.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,income,workclass_federal-gov,workclass_local-gov,workclass_never-worked,...,native_country_portugal,native_country_puerto-rico,native_country_scotland,native_country_south,native_country_taiwan,native_country_thailand,native_country_trinadad&tobago,native_country_united-states,native_country_vietnam,native_country_yugoslavia
7159,46.0,28281.0,13.0,0.0,0.0,40.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32121,26.0,375313.0,10.0,0.0,0.0,40.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22533,44.0,219591.0,10.0,0.0,0.0,40.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
11992,47.0,47496.0,9.0,0.0,0.0,42.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
14745,39.0,187089.0,10.0,0.0,0.0,40.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16274 entries, 7159 to 5314
Data columns (total 100 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   age                                        16274 non-null  float64
 1   fnlwgt                                     16274 non-null  float64
 2   education_num                              16274 non-null  float64
 3   capital_gain                               16274 non-null  float64
 4   capital_loss                               16274 non-null  float64
 5   hours_per_week                             16274 non-null  float64
 6   income                                     16274 non-null  int64  
 7   workclass_federal-gov                      16274 non-null  float64
 8   workclass_local-gov                        16274 non-null  float64
 9   workclass_never-worked                     16274 non-null  float64
 10  workclass_private       

In [ ]:
#linear regression for linearity check
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler


X = df_encoded.drop('income', axis=1)
y = df_encoded['income']


# Splitting in trainings- and testdata
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# model a linear regression
model = LinearRegression()
model.fit(X_train_std, y_train)

# predict on trainingsdata
y_train_pred = model.predict(X_train_std)
y_test_pred = model.predict(X_test_std)

# calculate the R^2-value
r2 = r2_score(y_train, y_train_pred)
print(f'R^2 Score on training data: {r2}')
r2_test = r2_score(y_test, y_test_pred)
print(f'R^2 Score on test data: {r2_test}')

R^2 Score on training data: 0.31403927613887117
R^2 Score on test data: 0.29731278558427754


29.05% of the variance gets explained.

The class balance gets checked. If the imbalance is too big, SMOTE will be used.

In [ ]:
class_distribution = df_encoded['income'].value_counts()
print(class_distribution)

income
0    13100
1     3174
Name: count, dtype: int64


In [ ]:
class_distribution_percentage = df_encoded['income'].value_counts(normalize=True) * 100
print(class_distribution_percentage)

income
0    80.496497
1    19.503503
Name: proportion, dtype: float64


It can be seen, that the classes are very imbalanced. Therefore, SMOTE will be used. SMOTE (Synthetic Minority Oversampling Technique) creats new instances from minority cases, which are already existing. SMOTE doesn`t affect the number of the majority cases.

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
X = df_encoded.drop(columns='income')
y = df_encoded['income']

In [ ]:
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

# SMOTE
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

# combine resampled trainingsdate and testdata
X_combined = np.vstack((X_train_res, X_test))
y_combined = np.hstack((y_train_res, y_test))

df_encoded_combined = pd.DataFrame(X_combined, columns=X.columns)
df_encoded_combined['income'] = y_combined

print(df_encoded_combined.head())
print('Shape of df_encoded_combined:', df_encoded_combined.shape)

Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7ca7a2d35e10>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.10/dist-packages/threadpoolctl.py", line 1175, in _make_controller_from_path
    lib_controller = controller_class(
  File "/usr/local/lib/python3.10/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
  File "/usr/lib/python3.10/ctypes/__init__.py", line 374, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: /usr/local/lib/python3.10/dist-packages/scipy.libs/libopenblasp-r0-01191904.3.27.so: cannot open shared object file: No such file or directory


Before OverSampling, counts of label '1': 2249
Before OverSampling, counts of label '0': 9142 

After OverSampling, counts of label '1': 9142
After OverSampling, counts of label '0': 9142
    age    fnlwgt  education_num  capital_gain  capital_loss  hours_per_week  \
0  46.0  146919.0            9.0           0.0           0.0            40.0   
1  55.0  200235.0            9.0           0.0           0.0            50.0   
2  33.0  295589.0           10.0           0.0           0.0            35.0   
3  50.0  116933.0            9.0           0.0           0.0            33.0   
4  22.0  198244.0            9.0           0.0           0.0            39.0   

   workclass_federal-gov  workclass_local-gov  workclass_never-worked  \
0                    0.0                  0.0                     0.0   
1                    0.0                  0.0                     0.0   
2                    0.0                  0.0                     0.0   
3                    0.0               

In [ ]:
print("Shape of X_train_res:", X_train_res.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of X_combined:", X_combined.shape)
print("Shape of y_train_res:", y_train_res.shape)
print("Shape of y_test:", y_test.shape)
print("Shape of y_combined:", y_combined.shape)

Shape of X_train_res: (18284, 99)
Shape of X_test: (4883, 99)
Shape of X_combined: (23167, 99)
Shape of y_train_res: (18284,)
Shape of y_test: (4883,)
Shape of y_combined: (23167,)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.decomposition import KernelPCA, PCA

In [ ]:
target_column = 'income'

y = df_encoded_combined[target_column]
X = df_encoded_combined.drop(columns=[target_column])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

###Test-dataset transformation###

X_test_final_std = scaler.transform(X_test_final)

# LDA
#lda = LinearDiscriminantAnalysis()
#lda.fit(X_train_std, y_train)
#y_pred_lda = lda.predict(X_test_std)
#accuracy_lda = accuracy_score(y_test, y_pred_lda)
#print(f'Accuracy of LDA : {accuracy_lda}')

#LDA 0.7976687293135702


#from sklearn.decomposition import PCA
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


# PCA (keeping enough components to explain 80% of the variance of the data)
#pca = PCA(n_components=0.90)
#X_train_pca = pca.fit_transform(X_train_std)
#X_test_pca = pca.transform(X_test_std)

#LDA
#lda = LinearDiscriminantAnalysis()
#lda.fit(X_train_pca, y_train)
#y_pred_lda = lda.predict(X_test_pca)

#PCA and LDA: 0.7976687293135702



#kpca = KernelPCA(n_components=100, kernel='poly')
#X_train_kpca = kpca.fit_transform(X_train_std)
#X_test_kpca = kpca.transform(X_test_std)

#LDA on kPCA-transformed data
#lda = LinearDiscriminantAnalysis()
#lda.fit(X_train_kpca, y_train)
#y_pred_lda = lda.predict(X_test_kpca)

#accuracy_lda = accuracy_score(y_test, y_pred_lda)
#print(f'Accuracy of LDA after kPCA: {accuracy_lda}')

#kPCA and LDA: 0.7999712188804144 kernel'rbf' components 1200 /same for 100 components
#kPCA and LDA: 0.8005468412721255 kernel 'poly' components 1200 /same for 100 components


The combination od kPCA and LDA performs best, therefore it will be used on the data.

In [ ]:
# KernelPCA
kpca = KernelPCA(n_components=1200, kernel='poly')
X_train_kpca = kpca.fit_transform(X_train_std)
X_test_kpca = kpca.transform(X_test_std)

#LDA on kPCA-transformed data
lda = LinearDiscriminantAnalysis()
lda.fit_transform(X_train_kpca, y_train)
y_pred_lda = lda.predict(X_test_kpca)

# Genauigkeit berechnen
accuracy = accuracy_score(y_test, y_pred_lda)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 86.82%


In [ ]:
#transformation traindata into dataframe
X_train_lda = lda.transform(X_train_kpca)
df_train_lda = pd.DataFrame(X_train_lda, columns=[f'lda_{i+1}' for i in range(X_train_lda.shape[1])], index=y_train.index)
df_train_lda[target_column] = y_train

# Transform test-datat into dataframe
X_test_lda = lda.transform(X_test_kpca)
df_test_lda = pd.DataFrame(X_test_lda, columns=[f'lda_{i+1}' for i in range(X_test_lda.shape[1])], index=y_test.index)
df_test_lda[target_column] = y_test

# combination of both dataframes
df_encoded = pd.concat([df_train_lda, df_test_lda], axis=0)

print('\nFinal training dataset:')
print(df_encoded.head())


Final training dataset:
          lda_1  income
13172  3.808526       1
2394  -1.856677       0
10935  1.368470       0
10454 -1.854464       0
5918  -0.873393       0


In [ ]:
df_encoded.shape

(23167, 2)

In [ ]:
#kPCA on trainingsdata
X_test_kpca = kpca.transform(X_test_final_std)  # kpca fitted on trainingsdata

#LDA on testdata
X_test_lda = lda.transform(X_test_kpca)  # lda wurde bereits auf den Trainingsdaten gefittet

#transformatio into dataframe and concat target
df_test_lda = pd.DataFrame(X_test_lda, columns=['lda_1'], index=y_test_encoded_df.index)  #l1 as column
df_test_lda['income'] = y_test_encoded_df.values  #naming target income

print('\nFinal test-dataset:')
print(df_test_lda.head())

          lda_1  income
7057  -1.319791       0
25716 -2.273363       0
7060   0.556461       1
16247 -1.585872       0
16991 -1.672495       0


In [ ]:
#download
from google.colab import files
df_encoded.to_csv('TrainSet_klassification.csv', index=False)
df_test_lda.to_csv('TestSet_klassification.csv', index=False)
files.download('TrainSet_klassification.csv')
files.download('TestSet_klassification.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>